Phân tích Đề bài & LogicMục tiêu: Tìm $x$ sao cho $a \cdot x \equiv 1 \pmod m$.
Điều kiện: Không dùng thư viện, không dùng đệ quy, phải xử lý số âm và ngoại lệ.Logic cốt lõi (Core Logic):Để tìm nghịch đảo, ta cần giải phương trình Bezout: $a \cdot x + m \cdot y = \gcd(a, m)$.Nếu $\gcd(a, m) = 1$, thì $x$ chính là nghịch đảo.
Quá trình này yêu cầu duy trì 3 dãy số song song trong vòng lặp:
Dãy số dư ($r$): Giảm dần từ $a, m$ về 0 (theo thuật toán chia Euclid).
Dãy hệ số ($x$): Biến đổi theo cùng quy luật của $r$ để thoả mãn phương trình.
Dãy hệ số ($y$): (Tương tự $x$, nhưng thực tế trong RSA/Modular Inverse ta thường không cần dùng đến $y$ ở kết quả cuối cùng, nhưng vẫn cần tính hoặc hiểu ngầm để thuật toán đúng).

In [2]:
class IntegerRepresentation:  #Biểu diễn số nguyên
    def __init__(self, a: int):
        self.a = a

    # Cần thiết cho hàm divmod() trong thuật toán
    def __divmod__(self, other):
        quotient = IntegerRepresentation(self.a // other.a)
        remainder = IntegerRepresentation(self.a % other.a)
        return quotient, remainder

    # Cần thiết để tính toán lại hệ số s và t (s1 - quotient(thương) * s2)
    def __mul__(self, other):
        if isinstance(other, IntegerRepresentation):
            return IntegerRepresentation(self.a * other.a)
        return IntegerRepresentation(self.a * other) # Xử lý khi nhân với số thường

    def __sub__(self, other):
        return IntegerRepresentation(self.a - other.a)
    
    # Điều kiện dừng của vòng lặp while
    def is_zero(self) -> bool:
        return self.a == 0

    # Khởi tạo giá trị ban đầu cho s1, t1
    def get_zero_element(self):
        return IntegerRepresentation(0)

    def get_multiplicative_identity(self):
        return IntegerRepresentation(1)
    
    # Để in kết quả ra dễ đọc
    def __repr__(self):
        return f"{self.a}"


# Trích từ algorithm.py
def extendedEuclidAlgorithm(a, b):
    """
    Trả về (gcd, s, t) sao cho s*a + t*b = gcd(a,b)
    """
    s1, s2 = a.get_multiplicative_identity(), a.get_zero_element()
    t1, t2 = a.get_zero_element() , a.get_multiplicative_identity()
    while not b.is_zero():

# a = b.q+r
        quotient, remainder = divmod(a, b)
        a = b
        b = remainder

#r = s.A x t.B
        s1, s2 = s2, s1 - quotient*s2
        t1, t2 = t2, t1 - quotient*t2
    return a, s1, t1


# --- Phần code thực thi (Main) ---

# Ví dụ: Tìm nghịch đảo của 123 (3 chữ số) trong module 17 (2 chữ số)
so_can_nghich_dao = 992  # a
module_so = 39           # b

# 1. Chuyển đổi sang đối tượng IntegerRepresentation
obj_a = IntegerRepresentation(so_can_nghich_dao)
obj_b = IntegerRepresentation(module_so)

# 2. Chạy thuật toán EEA
ket_qua_gcd, he_so_s, he_so_t = extendedEuclidAlgorithm(obj_a, obj_b)

# 3. Xử lý kết quả để tìm nghịch đảo
uoc_chung = ket_qua_gcd.a
nghich_dao_tho = he_so_s.a # Đây là giá trị s tìm được
print('mssv: 3993, Trần Đức Lĩnh')
print(f"GCD({so_can_nghich_dao}, {module_so}) = {uoc_chung}")

if uoc_chung != 1:
    print(f"Không tồn tại nghịch đảo vì {so_can_nghich_dao} và {module_so} không nguyên tố cùng nhau.")
else:
    # Đôi khi s ra số âm, ta cần đưa về số dương trong module b
    # Python xử lý % số âm rất tốt, nhưng công thức tổng quát là (s % b + b) % b
    nghich_dao_duong = (nghich_dao_tho % module_so)
    
    print(f"Hệ số Bezout s (nghịch đảo thô): {nghich_dao_tho}")
    print(f"Nghịch đảo của {so_can_nghich_dao} mod {module_so} là: {nghich_dao_duong}")
    
    # Kiểm tra lại: (123 * nghịch_dao) % 17 phải bằng 1
    kiem_tra = (so_can_nghich_dao * nghich_dao_duong) % module_so
    print(f"Kiểm tra lại: {so_can_nghich_dao} * {nghich_dao_duong} % {module_so} = {kiem_tra}")

mssv: 3993, Trần Đức Lĩnh
GCD(992, 39) = 1
Hệ số Bezout s (nghịch đảo thô): -16
Nghịch đảo của 992 mod 39 là: 23
Kiểm tra lại: 992 * 23 % 39 = 1
